# Algorithme de Fusion

In [ ]:
#import des packages requis
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 

In [ ]:
def only_longueur_1(dico):
    '''prend un dico en entrée et regarde si chacun de ses éléments n'est que des listes de longueur 1 '''
    for j in dico.values():
        if len(j) != 1:
            return False
    return True

In [ ]:
def fusion_de_haies(haies_):
    '''Prend un dataframe de haies en entrée et renvoie un dataframe après fusions des haies
    A noté : les ids des haies fusionnées sont > 1000000'''
    haies = haies_.copy()

    #On remplit notre dictionnaire
    dico_extremite = {}
    for i, haie in haies.iterrows():
        for x1, y1 in [(haie['X1'], haie['Y1']),(haie['X2'], haie['Y2'])]:
            if (x1, y1) in dico_extremite:
                dico_extremite[(x1, y1)].append(haie['id'])
            else:
                dico_extremite[(x1, y1)] = [haie['id']]

    #On définit les nouveaux indices
    new_index = 1000000
    
    #On parcourt les extrêmitées contenues dans le dictionnaire
    while dico_extremite!={} and not only_longueur_1(dico_extremite):
        extremités,haies_ids = list(dico_extremite.items())[0]
        x1_mid,x2_mid = extremités

        #On regarde si l'extremité est reliée à au moins 2 haies
        if len(haies_ids) > 1 :
            longueur = 0
            angle = 0
            #On stocke les indices des deux haies qui communiquent avec l'extrêmité courante
            index_1 = haies[haies['id'] == haies_ids[0]].index[0]
            index_2 = haies[haies['id'] == haies_ids[1]].index[0]
            #On stocke les deux haies qui touchent l'extrêmité courante
            haie_1 = haies.loc[index_1]
            haie_2 = haies.loc[index_2]
            #On stocke row et col de la haie 1 dans les variables r et c 
            r = haie_1['row']
            c = haie_1['col']

            #On regarde si l'angle entre les deux haies est suffisamment petit
            ang1 = haie_1['Ang']
            ang2 = haie_2['Ang']
            if abs(ang1 - ang2) < 0.175:
                #On construit la nouvelle haie, il y a 4 cas de figure :
                #1er cas :
                if (haie_1['X1'], haie_1['Y1']) == extremités and (haie_2['X1'],haie_2['Y1']) == extremités :
                    new_haie_x1, new_haie_y1 = haie_2['X2'], haie_2['Y2']
                    new_haie_x2, new_haie_y2 = haie_1['X2'], haie_1['Y2']

                    if (new_haie_x1,new_haie_y1) in dico_extremite :
                        dico_extremite[(new_haie_x1,new_haie_y1)].remove(haies_ids[1])
                    if (new_haie_x2,new_haie_y2) in dico_extremite:
                        dico_extremite[(new_haie_x2,new_haie_y2)].remove(haies_ids[0])

                #2eme cas :
                elif (haie_1['X1'], haie_1['Y1']) == extremités and (haie_2['X2'],haie_2['Y2']) == extremités :
                    new_haie_x1, new_haie_y1 = haie_1['X2'], haie_1['Y2']
                    new_haie_x2, new_haie_y2 = haie_2['X1'], haie_2['Y1']

                    if (new_haie_x1,new_haie_y1) in dico_extremite : 
                        dico_extremite[(new_haie_x1,new_haie_y1)].remove(haies_ids[0])
                    if (new_haie_x2,new_haie_y2) in dico_extremite : 
                        dico_extremite[(new_haie_x2,new_haie_y2)].remove(haies_ids[1])

                #3eme cas :
                elif (haie_1['X2'], haie_1['Y2']) == extremités and (haie_2['X1'],haie_2['Y1']) == extremités :
                    new_haie_x1, new_haie_y1 = haie_1['X1'], haie_1['Y1']
                    new_haie_x2, new_haie_y2 = haie_2['X2'], haie_2['Y2']

                    if (new_haie_x1,new_haie_y1) in dico_extremite :
                        dico_extremite[(new_haie_x1,new_haie_y1)].remove(haies_ids[0])
                    if (new_haie_x2,new_haie_y2) in dico_extremite :
                        dico_extremite[(new_haie_x2,new_haie_y2)].remove(haies_ids[1])

                #4eme cas :
                elif (haie_1['X2'], haie_1['Y2']) == extremités and (haie_2['X2'],haie_2['Y2']) == extremités :
                    new_haie_x1, new_haie_y1 = haie_1['X1'], haie_1['Y1']
                    new_haie_x2, new_haie_y2 = haie_2['X1'], haie_2['Y1']

                    if (new_haie_x1,new_haie_y1) in dico_extremite :
                        dico_extremite[(new_haie_x1,new_haie_y1)].remove(haies_ids[0])
                    if (new_haie_x2,new_haie_y2) in dico_extremite :
                        dico_extremite[(new_haie_x2,new_haie_y2)].remove(haies_ids[1])

                #On calcule la longueur et l'angle de cette nouvelle haie
                longueur = haie_1['Lng'] + haie_2['Lng']
                angle = (haie_1['Ang'] + haie_2['Ang'])/2

                #On ajoute la nouvelle haie au dataset 'haies'
                new_row = pd.DataFrame({'row': [r], 'col': [c], 'id': [new_index], 'X1': [new_haie_x1], 'Y1': [new_haie_y1], 'X2': [new_haie_x2], 'Y2': [new_haie_y2], 'Lng': [longueur], 'Ang': [angle]})
                haies = pd.concat([haies, new_row], ignore_index=True)


                #On remplace l'id de la haie_1 et l'id de la haie_2 par new_index dans les extremités de la nouvelle haie
                if (new_haie_x1,new_haie_y1) in dico_extremite :
                    dico_extremite[(new_haie_x1,new_haie_y1)].append(new_index)

                if (new_haie_x2,new_haie_y2) in dico_extremite :
                    dico_extremite[(new_haie_x2,new_haie_y2)].append(new_index)

                #On supprime les haie_1 et haie_2 du dataset 'haies'
                haies = haies.drop(haies[haies['id'] == haies_ids[0]].index[0])
                haies = haies.drop(haies[haies['id'] == haies_ids[1]].index[0])
                #On met l'index à jour
                new_index+=1
    
            #Une fois qu'on est sorti de toutes les boucles, on supprime l'extremité concernée
            del dico_extremite[(x1_mid,x2_mid)]
        else:
            #Si l'extrémité n'a qu'une seule haie on la garde quand même, mais juste on la met à la fin du dico
            del dico_extremite[(x1_mid,x2_mid)]
            dico_extremite[(x1_mid,x2_mid)] = [haies_ids[0]]

    #On retourne le dataset modifié
    return pd.DataFrame(haies)

# Fusion des haies

In [ ]:
# On importe le dataset "haies"
haies = pd.read_csv('1_haies.csv', delimiter=',',index_col=0)
# On fusionne les haies
haie_fusion = fusion_de_haies(haies)
# On télécharge le dataset contenant les haies fusionnées
haie_fusion.to_csv('3_haie_fusion.csv')
# On créer 2 dataset : l'un avec les haies qui ont étés fusionnées, l'autre avec les haies qui n'ont pas bougés
haie_vraiment_fusionnees = haie_fusion[haie_fusion['id'] > 100000]
haie_non_fusionnees = haie_fusion[haie_fusion['id'] < 100000]
# On télécharge ces deux dataset 
haie_vraiment_fusionnees.to_csv('3_haie_vraiment_fusionnees.csv')
haie_non_fusionnees.to_csv('3_haie_non_fusionnees.csv')

# Fusion des canaux

In [ ]:
# On importe le dataset "canaux"
canaux = pd.read_csv('1_canaux.csv', delimiter=',',index_col=0)
# On fusionne les canaux à l'aide de la fonction fusion_de_haies (ça marche exactement pareil)
canaux_fusion = fusion_de_haies(canaux)
# On télécharge le dataset contenant les canaux fusionnées
canaux_fusion.to_csv('3_canaux_fusion.csv')
# On créer 2 dataset : l'un avec les canaux qui ont étés fusionnées, l'autre avec les canaux qui n'ont pas bougés
canaux_vraiment_fusionnees = canaux_fusion[canaux_fusion['id'] > 100000]
canaux_non_fusionnees = canaux_fusion[canaux_fusion['id'] < 100000]
# On télécharge ces deux dataset 
canaux_vraiment_fusionnees.to_csv('3_canaux_vraiment_fusionnees.csv')
canaux_non_fusionnees.to_csv('3_canaux_non_fusionnees.csv')